Date created: July 13, 2022

---

## Project 2: Flow Over a Flat Plate
**Authors:** Tyler Reiser, Mason F., Brian Tan     
**Summary:** Modeling the flow over a flat plate.  
       
---

#### Finding the coupled system
We have two coupled, ordinary differential equations. We want $G$, but we need to know $F$ to solve for $G$. So there are two equations. Writing it as a system is a little more difficult, so let's split it up. 

$F$ is a third-order differential equation with three given boundary conditions. So this is a **three-point boundary valued problem**. These are really tough to solve, unless we can turn it into an initial value problem. We can solve IVP easily with RK4. Look at [1] for reference on how to transform the higher-order differential equation as a coupled system of first-order differential equations. This is one of the algorithms learned in differential equations.

Let the coupled system of first order differential equations be,

$x' = y$   
$y' = z$    
$z' = - \frac{1}{2} xz$   

and let the initial conditions be,   
$x(0) = F(0) = 0$,   
$y(0) = F'(0) = 0$, and    
$z(0) = F''(0) = u$. 

Note that we are making a guess at the initial conditions here. Since the success of the algorithm relies on this guess, we need to "do our homework" on the problem and make sure this is an educated guess. This is the analysis of the numerics. 
>**TEAM!** Lets add something here about *WHY* these are good guesses. 

Now, consider the boundary-values given by the problem. Notice that one point is at infinity; this implies that the first two boundary-values are very close to eachother when compared to their distance from the point at infinity. Because of this, I think it's best to integrate from left-to-right. The step size is given to us as $0.1$.
        
---

#### Fourth-order Runge Kutta Method
Adding more equations to the RK4 algorithm doesn't change the code from project 1 in too many ways. It looks big and confusing but it just has more variables. Note that this is written without loops to avoid type-checks. The algorithm is in one function that moves our initial guesses across time. Since we have a system of differential equations, at each step, we have to move each initial condition across time for every equation and RK4 tells us how we "average" the results of each output. The output is the coordinates of our initial conditions at the next step in time.

In [1]:
import numpy as np

def RK4(t, x, y, z, equation_1, equation_2, equation_3, dt):
    k1[0] = dt*equation_1(t, x, y, z)
    k1[1] = dt*equation_2(t, x, y, z)
    k1[2] = dt*equation_3(t, x, y, z)

    k2[0] = dt*equation_1(t + dt/2., x + k1[0]/2., y + k1[1]/2., z + k1[2]/2.)
    k2[1] = dt*equation_2(t + dt/2., x + k1[0]/2., y + k1[1]/2., z + k1[2]/2.)
    k2[2] = dt*equation_3(t + dt/2., x + k1[0]/2., y + k1[1]/2., z + k1[2]/2.)

    k3[0] = dt*equation_1(t + dt/2., x + k2[0]/2., y + k2[1]/2., z + k2[2]/2.)
    k3[1] = dt*equation_2(t + dt/2., x + k2[0]/2., y + k2[1]/2., z + k2[2]/2.)
    k3[2] = dt*equation_3(t + dt/2., x + k2[0]/2., y + k2[1]/2., z + k2[2]/2.)

    k4[0] = dt*equation_1(t + dt, x + k3[0], y + k3[1], z + k3[2])
    k4[1] = dt*equation_2(t + dt, x + k3[0], y + k3[1], z + k3[2])
    k4[2] = dt*equation_3(t + dt, x + k3[0], y + k3[1], z + k3[2])

    x = x + (1./6.)*(k1[0] + 2.*k2[0] + 2.*k3[0] + k4[0])
    y = y + (1./6.)*(k1[1] + 2.*k2[1] + 2.*k3[1] + k4[1])
    z = z + (1./6.)*(k1[2] + 2.*k2[2] + 2.*k3[2] + k4[2])

    return np.array([x, y, z])

---
#### Parameters
Now we define the variables going through the algorithm and define the coupled system as three functions. This is pretty straight forward.

In [10]:
# time scale
t0 = 0.
dt = 0.1 # given step size
tmax = 2.
vector_t = np.arange(t0, tmax, dt)
n = len(vector_t)

u = 1

# setting initial conditions
rk4_data = np.zeros((n, 3))
rk4_data[0, 0] = 0
rk4_data[0, 1] = 0
rk4_data[0, 2] = u

# storage
k1 = np.array([0., 0., 0.])
k2 = np.array([0., 0., 0.])
k3 = np.array([0., 0., 0.])
k4 = np.array([0., 0., 0.])

# coupled system from above
def equation_1(t, x, y, z):
    return y

def equation_2(t, x, y, z):
    return z

def equation_3(t, x, y, z):
    return -1/2*(x*z)

---
#### Creating a data stream
The time-scale is defined above. We want to move the initial conditions across time, so a single loop is used. This is because this is an **iterative method**. Slicing is used to print the first and last four values of the data stream.

In [11]:
for i in range(n-1): 
    rk4_data[i+1, :] = RK4(vector_t[i], rk4_data[i, 0], rk4_data[i, 1], rk4_data[i, 2], equation_1, equation_2, equation_3, dt)

# printing some data
print("Take a look at our data stream: \n\n", rk4_data[:4, :])
print("  ...  ")
print(rk4_data[n-4:, :])

Take a look at our data stream: 

 [[0.         0.         1.        ]
 [0.005      0.09999792 0.99991667]
 [0.01999875 0.19996667 0.99933358]
 [0.04499    0.29983137 0.99775279]]
  ...  
[[1.23904602 1.47680926 0.71479817]
 [1.39022614 1.54603501 0.66934346]
 [1.54809808 1.61061331 0.62195404]
 [1.71218849 1.67038306 0.5732856 ]]


---
### Shooting Method
The idea behing the *shooting method* is to check how well our guesses are by calculating the *boundary residual*. To do this, we compute the difference between the computed and given boundary values at a point. 

ie.   
actual boundary-values $ = U_R = [x, y, z]$   
guessed boundary-values $ = U = [x_0, y_0, z_0]$   
$res(U) = \theta(U) - U_R = 0 $ 

which is a **root-finding problem**, as long as the given boundary values straddle the root. For many reasons, *newton's method* is awesome, but to avoid finding the derivative, we can use **ridder's method**. 

But, let's think about **what we actually doing:**   
We are creating boxes with the size of our maximum time value. Within the box, we are defining a coordinate system based for a dynamical system and approximating the location of the system at x, y, z at a bunch of equally spaced times. These locations within the box are called *nodes*. Since we made a guess at the initial values, we are calculating a the residual at each one of these nodes to see the difference, then using that value to perform another interation. As long as our guess is "good," each iteration gets us a better approximation of the solution.


In [14]:
# residual
def boundary_residual(u):
    y = rk4_data[len(rk4_data) - 1]
    r = y[0] - 1.0
    return r

boundary_residual(u)

0.7121884940041685

#### Ridder's Method


In [15]:
import matplotlib.pyplot as plt

def ridders_method(f, x, y, epsilon=10e-10):
    f_x = f(x)
    if f_x == 0.: return x
    f_y = f(y)
    if f_y == 0.: return y
    
    for i in range(100):
        z = 0.5*(x + y)
        f_z 
    

# results
print('\nThe root value is ', x)
print('\nThe value of f(x) is ', y)
print('\nThe number of iterations to reach a solution is ',
      count, ' when x=', x0, 'is chosen as an initial condition')

NameError: name 'Newton' is not defined

---

### References

[1]:  [Couple Dif. Eq.](https://tutorial.math.lamar.edu/Classes/DE/SystemsDE.aspx)